# Bingo!

![game](http://s2.quickmeme.com/img/14/1494d43f99b56ab589c88a8138175c488be95d4c0b4dbdaebdea05d38790055f.jpg)

In [1]:
import pandas
import hashlib
import itertools
import operator

NUM_WINNERS = 2
SEED = 42

In [2]:
emails = pandas.read_csv("emails.csv")
print emails.count()
emails.head()

email    3
dtype: int64


,email
0,java@otus.ru
1,python@otus.ru
2,guido@otus.ru


# Naive approach

In [3]:
emails.sample(NUM_WINNERS, random_state=SEED)

,email
0,java@otus.ru
1,python@otus.ru


# Enter Pi

In [4]:
# http://www.cs.ox.ac.uk/people/jeremy.gibbons/publications/spigot.pdf
def pi_digits():
    """generator for digits of pi"""
    q, r, t, k, n, l = 1, 0, 1, 1, 3, 3
    while True:
        if 4 * q + r - t < n * t:
            yield n
            q, r, t, k, n, l = (10*q, 10*(r-n*t), t, k, (10*(3*q+r))/t-10*n, l)
        else:
            q, r, t, k, n, l = (q*k, (2*q+r)*l, t*l, k+1, (q*(7*k+2)+r*l)/(t*l), l+2)


_g = pi_digits()
print "".join([str(_g.next()) for _ in xrange(20)])

31415926535897932384


In [5]:
email2num = lambda email: int(hashlib.md5(email).hexdigest(), 16) % 10 ** 4

In [6]:
emails["num"] = emails["email"].map(email2num)
assert len(emails["email"].unique()) ==  len(emails["num"].unique())
emails.head()

,email,num
0,java@otus.ru,988
1,python@otus.ru,3267
2,guido@otus.ru,7764


In [7]:
class _Num(object):
    def __init__(self, n):
        self.n = n
        self.s = str(n)
        self.p = 0 # pointer in number string representation
        self.l = len(self.s)

    def move_p(self, d):
        if d == self.s[self.p]:
            self.p += 1
        else:
            self.p = 0

    
def find_nums_in_pi(nums, first_n=None):
    MAX_POS = 10 ** 6
    pi_gen = pi_digits()
    first_n = first_n if first_n is not None else len(nums)
    _nums = [_Num(n) for n in nums]
    nums_pos = {}
    for pos in itertools.count():
        if pos % 1000 == 0:
            print "Current Pi position: %s. Nums found: %s" % (pos, len(nums_pos))
        if pos == MAX_POS:
            raise RuntimeError("Circuit breaker!")
        d = str(pi_gen.next())
        for cur_num in _nums:
            cur_num.move_p(d)
            # whole number found
            if cur_num.p == cur_num.l:
                nums_pos[cur_num.n] = pos - cur_num.l + 1
                # found enough numbers
                if len(nums_pos) == first_n:
                    return nums_pos
                # create new search array without found number
                _nums = [num for num in _nums if num.n != cur_num.n]
                break

In [8]:
%%time
num_pos = find_nums_in_pi(emails["num"].tolist(), first_n=NUM_WINNERS)
print num_pos

Current Pi position: 0. Nums found: 0
Current Pi position: 1000. Nums found: 0
{7764: 1953, 988: 1536}
CPU times: user 319 ms, sys: 12.4 ms, total: 332 ms
Wall time: 323 ms


In [9]:
for n, p in sorted(num_pos.iteritems(), key=operator.itemgetter(1)):
    r = emails.loc[emails['num'] == n]
    print "%s -> %s" % (r["email"].iloc[0], p)

java@otus.ru -> 1536
guido@otus.ru -> 1953


![winner](http://podrobnosti.ua/media/pictures/2015/10/30/thumbs/740x415/v-etu-pjatnitsu-pole-chudes-vyhodit-v-efir-rovno-25-god-podrjad-kadr-iz-video_rect_8d40de7cfa137c6ac9d9fcd7378620c0.jpg)